In [2]:
# Library
import sys, os
sys.path.append(os.path.abspath('..'))

from hdf5_loader import StockDatasetHDF5
from config import *

import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime, timedelta
import os
import gc
import re
import h5py
import random
import importlib
import shutil

import torch
import torch.nn as nn
from torch.utils.data import IterableDataset, DataLoader

In [3]:
import gymnasium as gym
from gymnasium import spaces

In [222]:
hdf5_train = StockDatasetHDF5(
    ticker_list=[
    'AAPL',  # Apple Inc.
    'MSFT',  # Microsoft Corporation
],
    date_range=[ST, ED-timedelta(52)]
)

In [223]:
UNIT_TS = {
    '1m': 60,
    '5m': 60*5,
    '30m': 60*30,
    '1d': 60*60*24,
    '1w': 60*60*24*7
}

In [224]:
hz_dim = {hz:6 for hz in THZ}
targ_hz = '5m'

temp = dict()
for hz in THZ:
    df = hdf5_train[0][hz]
    df['timestamp'] = df.index.astype('int64') // 10**9
    # add global feature
    
     
    temp[hz] = df.to_numpy()

In [225]:
# make sample
curidx = {hz:0 for hz in THZ}
for i in [6330]:
    targ_time = temp[targ_hz][i, 7]
    # indexing
    for hz in THZ:
        while True:
            if temp[hz][curidx[hz], 7] + UNIT_TS[hz] > targ_time + UNIT_TS[targ_hz]: break
            else: curidx[hz] += 1
    
    # split
    result = dict()
    for hz in THZ:
        if (diff := curidx[hz] - hz_dim[hz]) >= 0:
            result[hz] = temp[hz][diff:curidx[hz]]
        else:
            result[hz] = np.pad(temp[hz][:curidx[hz]], pad_width=((-diff, 0),(0,0)), mode='constant', constant_values=0)
    
    # add local feature
    
    # print(result)
    

In [227]:
for hz in THZ:
    print(result[hz].shape)
    for ts in result[hz][:,7]:
        print(datetime.fromtimestamp(ts-60*60*9))

(6, 8)
2015-04-30 09:29:00
2015-04-30 09:30:00
2015-04-30 09:31:00
2015-04-30 09:32:00
2015-04-30 09:33:00
2015-04-30 09:34:00
(6, 8)
2015-04-30 09:05:00
2015-04-30 09:10:00
2015-04-30 09:15:00
2015-04-30 09:20:00
2015-04-30 09:25:00
2015-04-30 09:30:00
(6, 8)
2015-04-29 13:00:00
2015-04-29 13:30:00
2015-04-29 14:00:00
2015-04-29 14:30:00
2015-04-30 08:30:00
2015-04-30 09:00:00
(6, 8)
2015-04-22 00:00:00
2015-04-23 00:00:00
2015-04-24 00:00:00
2015-04-27 00:00:00
2015-04-28 00:00:00
2015-04-29 00:00:00
(6, 8)
2015-03-16 00:00:00
2015-03-23 00:00:00
2015-03-30 00:00:00
2015-04-06 00:00:00
2015-04-13 00:00:00
2015-04-20 00:00:00


---

In [ ]:
class StockTradingEnv(gym.Env):
    def __init__(self, render_mode=None):
        # self.
        
        self.observation_space = None
        self.action_space = spaces.Discrete(2)

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode
        
    def _get_obs(self):  # return s_t
        return {"agent": self._agent_location, "target": self._target_location}

    def _get_info(self):  # return aux. infos
        return {
            "distance": np.linalg.norm(
                self._agent_location - self._target_location, ord=1
            )
        }
        
    def reset(self):
        pass
    
    def step(self):
        pass
    
    def render(self):
        pass
    
    def close(self):
        pass